In [1]:
from __future__ import print_function, division
import os
import torch
from torch.optim.optimizer import Optimizer
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io
import torch.nn as nn
import torch.nn.functional as F

from skimage.color import rgb2gray

from torch.utils.tensorboard import SummaryWriter

import PIL
from PIL import Image

from google.colab import output

In [2]:
%mkdir results

In [3]:
!git clone https://github.com/PauliusMilmantas/Bakalaurinis_darbas

Cloning into 'Bakalaurinis_darbas'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 93 (delta 34), reused 86 (delta 27), pack-reused 0
Unpacking objects: 100% (93/93), done.


In [4]:
%cd /content/Bakalaurinis_darbas/Code

/content/Bakalaurinis_darbas/Code


In [5]:
import pandas as pd

duomenys = pd.read_csv('/content/Bakalaurinis_darbas/Code/diabetes.csv')
duomenys

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [6]:
import pandas as pd

class ORCDataset(Dataset):
  def __init__(self, root):
    self.root = root
    self.data = pd.read_csv(root)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    dat = {}
    dat['Pregnancies'] = self.data['Pregnancies'][idx]
    dat['Glucose'] = self.data['Glucose'][idx]
    dat['BloodPressure'] = self.data['BloodPressure'][idx]
    dat['BMI'] = self.data['BMI'][idx]
    dat['Age'] = self.data['Age'][idx]
    found_type = self.data['Outcome'][idx]

    return {'data': dat, 'class_name': found_type}

train_dataset = ORCDataset('/content/Bakalaurinis_darbas/Code/diabetes.csv')

In [7]:
class Net(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)        
        out = self.relu2(out)     
        out = self.fc3(out)
        return out

2 <- 7 <- 15 <- 5

In [8]:
network = Net(5, 15, 7, 2)
#network.cuda()
optimizer = torch.optim.SGD(network.parameters(), lr=0.0001, momentum=0.6)
criterion = nn.MSELoss().cuda()

print(network)

Net(
  (fc1): Linear(in_features=5, out_features=15, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=15, out_features=7, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=7, out_features=2, bias=True)
)


In [9]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
network.to(dev)
criterion.to(dev)
dev

device(type='cpu')

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cpu



In [11]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = len(train_dataset),shuffle=True)

In [12]:
dataiter = iter(train_loader)
itr = dataiter.next()

label = itr['class_name']
dat = itr['data']
print("Class name: {} Data: [Pregnancies: {} Glucose: {} BloodPressure: {} BMI: {} Age: {}]".format(label[0], dat['Pregnancies'][0], dat['Glucose'][0], dat['BloodPressure'][0], dat['BMI'][0], dat['Age'][0]))

Class name: 0 Data: [Pregnancies: 0 Glucose: 57 BloodPressure: 60 BMI: 21.7 Age: 67]


In [13]:
for data in train_loader:
  parsedData = []
  for dictIdx in range(len(dat)):
    parsedData.append([dat['Pregnancies'][dictIdx], dat['Glucose'][dictIdx], dat['BloodPressure'][dictIdx], dat['BMI'][dictIdx], dat['Age'][dictIdx]])

  print(torch.Tensor(parsedData))

tensor([[  0.0000,  57.0000,  60.0000,  21.7000,  67.0000],
        [  9.0000,  89.0000,  62.0000,  22.5000,  33.0000],
        [  0.0000, 180.0000,  78.0000,  59.4000,  25.0000],
        [  6.0000,  87.0000,  80.0000,  23.2000,  32.0000],
        [  1.0000,  82.0000,  64.0000,  21.2000,  23.0000]])


In [18]:
for data in train_loader:
  labels = torch.from_numpy(np.array(data['class_name']))
  dat = data['data']

  print(labels)
  print(labels[0].item())
  print(labels[1].item())
  print(labels[2].item())
  print(labels[3].item())

tensor([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,

In [24]:
def train(train_loader, epoch_amount, save_checkpoint = 10):
  network.eval()
  train_loss_hist = []

  checkpoint = save_checkpoint
  for epoch in range(epoch_amount):
    num_data_train = 0

    # TRAINING DATASET
    correct = 0
    for data in train_loader:
      labels = torch.from_numpy(np.array(data['class_name']))
      dat = data['data']

      lossSum = 0
      
      parsedData = []
      for dictIdx in range(len(labels)):
        parsedData.append(
            np.array([dat['Pregnancies'][dictIdx].float(), dat['Glucose'][dictIdx].float(), dat['BloodPressure'][dictIdx].float(), dat['BMI'][dictIdx].float(), dat['Age'][dictIdx].float()])
            )

      for idx in range(len(labels)):
        optimizer.zero_grad()

        print(parsedData[idx])
        outputs = network(parsedData[idx].flatten().float())

        print("Size of parsed data: {}".format(len(parsedData[idx])))
        print("Size of labels: {}".format(len(labels)))
        
        loss = criterion(torch.Tensor(parsedData[idx]), torch.Tensor(labels[0].item())) 
          
        print("Parsed data tensor: {} Labels tensor: {}".format(torch.Tensor(parsedData[idx]), torch.Tensor(labels[idx].item())))

        loss.backward()

        lossSum += loss.item()
        train_loss_hist.append(lossSum)

        optimizer.step()

    print("Epoch: {} Training loss: {}".format(epoch,train_loss_hist[len(train_loss_hist) - 1]))
    
    if(checkpoint == 0):
      torch.save(network.state_dict(), '/content/results/model.pth')
      torch.save(optimizer.state_dict(), '/content/results/optimizer.pth')

      checkpoint = save_checkpoint
    else:
      checkpoint -= 1

train(train_loader, 1)


[ 2.  82.  52.  28.5 25. ]


AttributeError: ignored